In [1]:
from classes.EventFactory import Event, EventFactoryfrom classes.TargetFactory import Target, TargetFactoryfrom classes.Segment import Segment#Initialize factoriesvideo_name = "accident1"event_factory = EventFactory.from_config(f"{video_name}/configs/event_factory_config.json")target_factory = TargetFactory.from_config(f"{video_name}/configs/target_factory_config.json")

In [ ]:
from ultralytics import YOLO#Initialize YOLO modelyolo_model = YOLO("yolov8s-world.pt")yolo_model.set_classes(target_factory.get_valid_labels())

In [ ]:
from modules import ExtractFrames#Read video and save keyframes to frames_cache; return pathsvideo_path = "traffic_accident.mov"frames_path = ExtractFrames.extract_frames(video_path, f"{video_name}/frames_cache", interval = 0.5)

In [ ]:
from modules import AnnotateFramesfrom tools.GetAllPaths import get_all_file_pathsannotated_frames_path = []frames_path = get_all_file_paths(f"{video_name}/frames_cache")#Run YOLO per frame to annotate ROIs; this may not improve accuracy and can harm scene understandingfor frame_path in frames_path:annotated_frames_path.append(AnnotateFrames.annotate_frame(frame_path=frame_path, yolo_model=yolo_model, output_dir=f"{video_name}/annotated_frames_cache", conf= 0.2))

In [ ]:
from modules import SegmentGeneration#Pack annotated frames into Segment listsegments = SegmentGeneration.generate_segments(f"{video_name}/annotated_frames_cache")

In [ ]:
from modules import SegmentGeneration#Pack unannotated frames into Segment listsegments = SegmentGeneration.generate_segments(f"{video_name}/frames_cache")

In [2]:
from modules import SegmentGenerationV2#Special pack: overlapping segments with 20% overlap。segments = SegmentGenerationV2.generate_segments(f"{video_name}/frames_cache", frames_per_segment=20, overlap_ratio= 0.2)

In [ ]:
from modules import SegmentAnalyze#Parallel analysis: faster but less accurate; merging needs framessegment_list = [segments[0]]SegmentAnalyze.batch_process_segments(target_factory=target_factory, event_factory=event_factory, segments=segment_list, video_name=video_name)

In [ ]:
from modules import SegmentAnalyzeV2#Serial analysis: slower but more accurate; merging doesn't need framessegment_list = segments[30:]SegmentAnalyzeV2.process_segments_serially(target_factory=target_factory, event_factory=event_factory, segments=segment_list, video_name=video_name, subsection_interval=60)

In [ ]:
from IPython.display import Image, displayimport base64# For testing: show segment imagesdef show_base64_image(base64_string):display(Image(data=base64.b64decode(base64_string)))for image in segments[31].get_images():show_base64_image(image)

In [3]:
from modules import FillSegments# Fill segments from JSON filesFillSegments.fill_segments(segments, event_factory, target_factory, video_name)

In [ ]:
# For testing: print segment infoprint(segments[3].events[2])

In [4]:
from classes.StoryTree import StoryPool# Build story poolstory_pool = StoryPool(segments)

In [ ]:
str(story_pool.print_story_lines())

In [ ]:
from modules import Load# Load as LangChain RAG documentsstory_docs = Load.load_stories(story_pool)event_docs = Load.load_events(segments)target_docs = Load.load_targets(segments)segment_docs = Load.load_segments(segments)

In [ ]:
for doc in story_docs:print(doc.page_content)

In [ ]:
from classes.VectorStore import VectorStorevector_store = VectorStore(segment_docs=segment_docs, event_docs=event_docs, target_docs=target_docs, story_docs=story_docs)

In [ ]:
result = vector_store.search_story("accident",top_k=1)

In [ ]:
print(result[0].metadata.get("whole_story"))